# Compute statistics for IEEE FS 7.3.1 run

In [4]:
import os
import numpy as np
import pandas as pd
from freesurfer.aparcstats2table import main as aparcstats2table
from freesurfer.asegstats2table import main as asegstats2table
from pathlib import Path

anonymizer = True

root_dir = Path.cwd()


def anondir(path: Path, prefix=root_dir) -> Path:
    """Anonymize a directory path by replacing user-specific parts with <root>."""
    if not anonymizer:
        return path
    path_str = str(path).replace(str(prefix), "<living-park>")
    return Path(path_str)


print(f"Root directory: {anondir(root_dir)}")
input_dir = root_dir / "fs-7.3.1_ieee"
print(f"Input directory: {anondir(input_dir)}")
output_dir = root_dir / "table_ieee"
print(f"Output directory: {anondir(output_dir)}")
output_dir.mkdir(parents=True, exist_ok=True)

Root directory: <living-park>
Input directory: <living-park>/fs-7.3.1_ieee
Output directory: <living-park>/table_ieee


In [5]:
dataset = pd.read_csv(
    root_dir
    / "cohort"
    / "vip_executions_stats_info_2visits_passed_qc_with_26_repetitions.csv"
)
cohort = pd.read_csv(root_dir / "cohort" / "cross-sectional_cohort_qced.csv")
subjects_in_cohort = cohort["PATNO_id"].unique()
dataset = dataset[dataset["subject_visit"].isin(subjects_in_cohort)]
print(f"Total subjects in dataset: {len(subjects_in_cohort)}")
print(f"Unique subjects in cohort: {dataset['subject'].nunique()}")

Total subjects in dataset: 534
Unique subjects in cohort: 267


In [ ]:
measurements = ["volume", "thickness", "area"]

rsv = dataset["subject_visit"].unique().tolist()


def compute_aseg_volume(subjects_dir, subjects, output_dir):
    tablefile = os.path.join(output_dir, "aseg.volume.tsv")
    args = [
        f"--sd={subjects_dir}",
        "--skip",
        "--subjects",
        *subjects,
        "--meas=volume",
        f"--tablefile={tablefile}",
    ]
    asegstats2table(args)


def compute_volume(subjects_dir, subjects, output_dir):
    for hemi in ["lh", "rh"]:
        tablefile = os.path.join(output_dir, f"{hemi}.aparc.volume.tsv")
        args = [
            f"--sd={subjects_dir}",
            "--skip",
            "--parc=aparc",
            f"--hemi={hemi}",
            "--subjects",
            *subjects,
            "--meas=volume",
            f"--tablefile={tablefile}",
        ]
        aparcstats2table(args)


def compute_thickness(subjects_dir, subjects, output_dir):
    for hemi in ["lh", "rh"]:
        tablefile = os.path.join(output_dir, f"{hemi}.aparc.thickness.tsv")
        args = [
            f"--sd={subjects_dir}",
            "--skip",
            "--parc=aparc",
            f"--hemi={hemi}",
            "--subjects",
            *subjects,
            "--meas=thickness",
            f"--tablefile={tablefile}",
        ]
        aparcstats2table(args)


def compute_area(subjects_dir, subjects, output_dir):
    for hemi in ["lh", "rh"]:
        tablefile = os.path.join(output_dir, f"{hemi}.aparc.area.tsv")
        args = [
            f"--sd={subjects_dir}",
            "--skip",
            "--parc=aparc",
            f"--hemi={hemi}",
            "--subjects",
            *subjects,
            "--meas=area",
            f"--tablefile={tablefile}",
        ]
        aparcstats2table(args)


compute_aseg_volume(input_dir, rsv, output_dir)
compute_volume(input_dir, rsv, output_dir)
compute_thickness(input_dir, rsv, output_dir)
compute_area(input_dir, rsv, output_dir)